In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import os
from tqdm import tqdm
from utils import df_series_strip

In [2]:
dataPath = "./Data/Raw/"
patchesDataFile = "NMS_patch_notes.xlsx"
wikipediaDataFile = "NMS_wikipedia_major_releases_notes.xlsx"
df = pd.read_excel(dataPath+patchesDataFile,index_col=0)
wikiDF = pd.read_excel(dataPath+wikipediaDataFile,index_col=0)

In [3]:
print(df.head(),"\n"*2,"="*20)
print(df.describe(),"\n"*2,"="*20)
print(df.info(),"\n"*2,"="*20)

           Title       Date  \
0  PC Patch 1.04 2016-08-18   
1  PC Patch 1.05 2016-08-19   
2  PC Patch 1.06 2016-08-20   
3  PC Patch 1.07 2016-09-02   
4  PC Patch 1.08 2016-09-04   

                                               Notes  Patch update  \
0  "PC Patch 1.04 - No Man's Sky"\n'Menu'\n'Lates...         False   
1  "PC Patch 1.05 - No Man's Sky"\n'Menu'\n'Lates...         False   
2  "PC Patch 1.06 - No Man's Sky"\n'Menu'\n'Lates...         False   
3  "PC Patch 1.07 - No Man's Sky"\n'Menu'\n'Lates...         False   
4  "PC Patch 1.08 - No Man's Sky"\n'Menu'\n'Lates...         False   

   Bug fix update  Scrape error                             Patch page url  
0            True         False  https://www.nomanssky.com//pc-patch-1-04/  
1            True         False  https://www.nomanssky.com//pc-patch-1-05/  
2            True         False  https://www.nomanssky.com//pc-patch-1-06/  
3            True         False  https://www.nomanssky.com//pc-patch-1-07/  
4      

In [4]:
print(wikiDF.head(),"\n"*2,"="*20)
print(wikiDF.describe(),"\n"*2,"="*20)
print(wikiDF.info(),"\n"*2,"="*20)

           Title       Date  \
1   Foundation\n 2016-11-01   
2   Pathfinder\n 2017-03-01   
3  Atlas Rises\n 2017-08-01   
4         Next\n 2018-07-01   
5    The Abyss\n 2018-10-01   

                                               Notes  \
1  User-made bases built from modular components,...   
2  Ability to share bases with other players New ...   
3  Addition of new narrative branch to main game'...   
4  Added support for  Xbox One  and  WeChat Full ...   
5  Expanded aquatic biomes features include new c...   

                                                 Ref  
1                                          [ 38 ] \n  
2                            [ 39 ] [ 40 ] [ 41 ] \n  
3                                   [ 43 ] [ 42 ] \n  
4  [ 44 ] [ 45 ] [ 46 ] [ 47 ] [ 48 ] [ 49 ] [ 50...  
5                                          [ 52 ] \n   

                      Date
count                   32
mean   2021-08-16 08:15:00
min    2016-11-01 00:00:00
25%    2020-01-09 00:00:00
50%    20

## Clean main patch notes dataframe

In [5]:
modDF = df_series_strip(df,target={"Title":r"-"},replacement=r" ",compare=True)

Before vs after
6        PS4 1.04 - 1.07
9      foundation-update
12     pathfinder-update
16    atlas-rises-update
24           next-update
Name: Title, dtype: object
6        PS4 1.04   1.07
9      foundation update
12     pathfinder update
16    atlas rises update
24           next update
Name: Title, dtype: object




In [6]:
modDF["Title"] = modDF["Title"].str.lower()
modDF

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
0,pc patch 1.04,2016-08-18,"""PC Patch 1.04 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-04/
1,pc patch 1.05,2016-08-19,"""PC Patch 1.05 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-05/
2,pc patch 1.06,2016-08-20,"""PC Patch 1.06 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-06/
3,pc patch 1.07,2016-09-02,"""PC Patch 1.07 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-07/
4,pc patch 1.08,2016-09-04,"""PC Patch 1.08 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-08/
...,...,...,...,...,...,...,...
234,worlds part ii 5.55,2025-02-12,"'Worlds Part II - 5.55'\n'February 12, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
235,expedition seventeen: titan,2025-02-12,"'Expedition Seventeen: Titan'\n'February 12, 2...",True,False,False,https://www.nomanssky.com//2025/02/expedition-...
236,worlds part ii 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
237,worlds part ii 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...


In [7]:
modDF.loc[(modDF["Title"].str.contains("update")==True)&(modDF["Title"].str.contains("development")==False)]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
9,foundation update,NaT,'Patch notes'\n'General Gameplay'\n'Fixed tech...,True,False,False,https://www.nomanssky.com//foundation-update
12,pathfinder update,NaT,'Patch Notes'\n'Audio'\n'50% more generative s...,True,False,False,https://www.nomanssky.com//pathfinder-update
16,atlas rises update,NaT,'Patch notes'\n'Generation'\n'Increased variet...,True,False,False,https://www.nomanssky.com//atlas-rises-update
24,next update,NaT,'Patch notes'\n'Third Person'\n'Third person c...,True,False,False,https://www.nomanssky.com//next-update/
40,abyss update,NaT,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/
42,visions update,NaT,'Patch notes'\n'Missions and Progression'\n'Fi...,True,False,False,https://www.nomanssky.com//visions-update/
45,beyond update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//beyond-update/
70,synthesis update,NaT,'Menu'\n'Latest patch notes'\n'Hello Games Clo...,True,False,False,https://www.nomanssky.com//synthesis-update/
76,living ship update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//living-ship-update/
81,exo mech update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//exo-mech-update/


In [8]:
modDF.loc[(modDF["Title"].str.contains("update")==True)&(modDF["Title"].str.contains("development")==False), "Patch update"] = True
modDF.loc[(modDF["Title"].str.contains("update")==True)&(modDF["Title"].str.contains("development")==False), "Bug fix update"] = False

In [9]:
modDF.loc[(modDF["Title"].str.contains("update")==True)&(modDF["Title"].str.contains("development")==True), "Patch update"] = False
modDF.loc[(modDF["Title"].str.contains("update")==True)&(modDF["Title"].str.contains("development")==True), "Bug fix update"] = True

In [10]:
modDF.loc[(modDF["Title"].str.contains("update")==True)&(modDF["Title"].str.contains("development")==False)]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
9,foundation update,NaT,'Patch notes'\n'General Gameplay'\n'Fixed tech...,True,False,False,https://www.nomanssky.com//foundation-update
12,pathfinder update,NaT,'Patch Notes'\n'Audio'\n'50% more generative s...,True,False,False,https://www.nomanssky.com//pathfinder-update
16,atlas rises update,NaT,'Patch notes'\n'Generation'\n'Increased variet...,True,False,False,https://www.nomanssky.com//atlas-rises-update
24,next update,NaT,'Patch notes'\n'Third Person'\n'Third person c...,True,False,False,https://www.nomanssky.com//next-update/
40,abyss update,NaT,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/
42,visions update,NaT,'Patch notes'\n'Missions and Progression'\n'Fi...,True,False,False,https://www.nomanssky.com//visions-update/
45,beyond update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//beyond-update/
70,synthesis update,NaT,'Menu'\n'Latest patch notes'\n'Hello Games Clo...,True,False,False,https://www.nomanssky.com//synthesis-update/
76,living ship update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//living-ship-update/
81,exo mech update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//exo-mech-update/


In [11]:
modDF.loc[(modDF["Title"].str.contains(r"[0-9]")==True)]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
0,pc patch 1.04,2016-08-18,"""PC Patch 1.04 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-04/
1,pc patch 1.05,2016-08-19,"""PC Patch 1.05 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-05/
2,pc patch 1.06,2016-08-20,"""PC Patch 1.06 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-06/
3,pc patch 1.07,2016-09-02,"""PC Patch 1.07 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-07/
4,pc patch 1.08,2016-09-04,"""PC Patch 1.08 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-08/
...,...,...,...,...,...,...,...
233,worlds part ii 5.54,2025-02-06,"'Worlds Part II - 5.54'\n'February 06, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
234,worlds part ii 5.55,2025-02-12,"'Worlds Part II - 5.55'\n'February 12, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
236,worlds part ii 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
237,worlds part ii 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...


In [12]:
modDF.loc[(modDF["Title"].str.contains(r"[0-9]")==True), "Patch update"] = False
modDF.loc[(modDF["Title"].str.contains(r"[0-9]")==True), "Bug fix update"] = True

In [13]:
modDF.loc[(modDF["Title"].str.contains(r"[0-9]")==True)]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
0,pc patch 1.04,2016-08-18,"""PC Patch 1.04 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-04/
1,pc patch 1.05,2016-08-19,"""PC Patch 1.05 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-05/
2,pc patch 1.06,2016-08-20,"""PC Patch 1.06 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-06/
3,pc patch 1.07,2016-09-02,"""PC Patch 1.07 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-07/
4,pc patch 1.08,2016-09-04,"""PC Patch 1.08 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-08/
...,...,...,...,...,...,...,...
233,worlds part ii 5.54,2025-02-06,"'Worlds Part II - 5.54'\n'February 06, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
234,worlds part ii 5.55,2025-02-12,"'Worlds Part II - 5.55'\n'February 12, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
236,worlds part ii 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
237,worlds part ii 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...


In [14]:
modDF

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
0,pc patch 1.04,2016-08-18,"""PC Patch 1.04 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-04/
1,pc patch 1.05,2016-08-19,"""PC Patch 1.05 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-05/
2,pc patch 1.06,2016-08-20,"""PC Patch 1.06 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-06/
3,pc patch 1.07,2016-09-02,"""PC Patch 1.07 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-07/
4,pc patch 1.08,2016-09-04,"""PC Patch 1.08 - No Man's Sky""\n'Menu'\n'Lates...",False,True,False,https://www.nomanssky.com//pc-patch-1-08/
...,...,...,...,...,...,...,...
234,worlds part ii 5.55,2025-02-12,"'Worlds Part II - 5.55'\n'February 12, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
235,expedition seventeen: titan,2025-02-12,"'Expedition Seventeen: Titan'\n'February 12, 2...",True,False,False,https://www.nomanssky.com//2025/02/expedition-...
236,worlds part ii 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...
237,worlds part ii 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...


In [15]:
df.loc[modDF["Patch update"]!=df["Patch update"]]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
11,Patch 1.13,2016-12-12,"'Patch 1.13'\n'December 12, 2016'\n'.'\n'Hello...",True,False,False,https://www.nomanssky.com//2016/12/patch-1-13/
15,Path Finder – Patch 1.24,2017-03-27,"'Path Finder – Patch 1.24'\n'March 27, 2017'\n...",True,False,False,https://www.nomanssky.com//2017/03/4511/
22,Atlas Rises Patch 1.37,2017-09-15,"'Atlas Rises Patch 1.37'\n'September 15, 2017'...",True,False,False,https://www.nomanssky.com//2017/09/atlas-rises...
23,Atlas Rises Patch 1.38,2017-10-03,"'Atlas Rises Patch 1.38'\n'October 03, 2017'\n...",True,False,False,https://www.nomanssky.com//2017/10/atlas-rises...
34,Development Update – 1.59,2018-09-06,"'Development Update – 1.59'\n'September 06, 20...",True,False,False,https://www.nomanssky.com//2018/09/development...
...,...,...,...,...,...,...,...
153,switch-update,NaT,NaN,False,True,True,https://www.nomanssky.com//switch-update/
160,Holiday 2022 Expeditions,2022-11-23,"'Holiday 2022 Expeditions'\n'November 23, 2022...",True,False,False,https://www.nomanssky.com//2022/11/holiday-202...
165,Fractal 4.13,2023-03-08,'Fixed an issue that caused incorrect options ...,True,False,False,https://www.nomanssky.com//2023/03/fractal-pat...
193,Holiday 2023 Expeditions,2023-12-08,"'Holiday 2023 Expeditions'\n'December 08, 2023...",True,False,False,https://www.nomanssky.com//2023/12/holiday-202...


In [16]:
df.loc[modDF["Bug fix update"]!=df["Bug fix update"]]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url
11,Patch 1.13,2016-12-12,"'Patch 1.13'\n'December 12, 2016'\n'.'\n'Hello...",True,False,False,https://www.nomanssky.com//2016/12/patch-1-13/
15,Path Finder – Patch 1.24,2017-03-27,"'Path Finder – Patch 1.24'\n'March 27, 2017'\n...",True,False,False,https://www.nomanssky.com//2017/03/4511/
22,Atlas Rises Patch 1.37,2017-09-15,"'Atlas Rises Patch 1.37'\n'September 15, 2017'...",True,False,False,https://www.nomanssky.com//2017/09/atlas-rises...
23,Atlas Rises Patch 1.38,2017-10-03,"'Atlas Rises Patch 1.38'\n'October 03, 2017'\n...",True,False,False,https://www.nomanssky.com//2017/10/atlas-rises...
34,Development Update – 1.59,2018-09-06,"'Development Update – 1.59'\n'September 06, 20...",True,False,False,https://www.nomanssky.com//2018/09/development...
...,...,...,...,...,...,...,...
153,switch-update,NaT,NaN,False,True,True,https://www.nomanssky.com//switch-update/
160,Holiday 2022 Expeditions,2022-11-23,"'Holiday 2022 Expeditions'\n'November 23, 2022...",True,False,False,https://www.nomanssky.com//2022/11/holiday-202...
165,Fractal 4.13,2023-03-08,'Fixed an issue that caused incorrect options ...,True,False,False,https://www.nomanssky.com//2023/03/fractal-pat...
193,Holiday 2023 Expeditions,2023-12-08,"'Holiday 2023 Expeditions'\n'December 08, 2023...",True,False,False,https://www.nomanssky.com//2023/12/holiday-202...


In [17]:
modDF["Expedition"] = False
modDF.loc[modDF["Title"].str.contains("expedition")==True,"Expedition"] = True
modDF = df_series_strip(modDF,target={"Title":r"expedition [a-zA-Z]+[:]{1}"},compare=True)

Before vs after
124    expedition three: cartographers
129         expedition four: emergence
142        expedition seven: leviathan
149         expedition eight: polestar
174        expedition ten: singularity
Name: Title, dtype: object
124     cartographers
129         emergence
142         leviathan
149          polestar
174       singularity
Name: Title, dtype: object




In [18]:
modDF = df_series_strip(modDF,target={"Title":r"update"},compare=True)
modDF = df_series_strip(modDF,target={"Title":r"the "},compare=True)

Before vs after
9             foundation update
12            pathfinder update
16           atlas rises update
24                  next update
33    development update – 1.58
Name: Title, dtype: object
9             foundation 
12            pathfinder 
16           atlas rises 
24                  next 
33    development  – 1.58
Name: Title, dtype: object


Before vs after
220                                           the cursed
221                                the cursed patch 5.21
222                                the cursed patch 5.22
223    cross save // playstation 5 pro // the normand...
224    cross save // playstation 5 pro // the normand...
Name: Title, dtype: object
220                                               cursed
221                                    cursed patch 5.21
222                                    cursed patch 5.22
223    cross save // playstation 5 pro // normandy re...
224    cross save // playstation 5 pro // normandy re...
Name: Title, dtype: objec

In [19]:
modDF["Title"] = modDF["Title"].str.strip()

In [20]:
modDF.loc[modDF["Title"].str.contains("cursed")==True]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Expedition
220,cursed,2024-10-23,"'Expedition Sixteen: The Cursed'\n'October 23,...",True,False,False,https://www.nomanssky.com//2024/10/expedition-...,True
221,cursed patch 5.21,2024-10-25,"""The Cursed Patch 5.21 - No Man's Sky""\n'Menu'...",False,True,False,https://www.nomanssky.com//2024/10/the-cursed-...,False
222,cursed patch 5.22,2024-10-30,"""The Cursed Patch 5.22 - No Man's Sky""\n'Menu'...",False,True,False,https://www.nomanssky.com//2024/10/the-cursed-...,False


## Clean wikipedia major releases notes dataframe

In [21]:
modWiki = df_series_strip(wikiDF,target={"Title":r"-"},replacement=r" ",compare=True)
modWiki["Title"] = modWiki["Title"].str.lower()
modWiki

Before vs after
10    Exo-Mech\n
Name: Title, dtype: object
10    Exo Mech\n
Name: Title, dtype: object




,Title,Date,Notes,Ref
1,foundation\n,2016-11-01,"User-made bases built from modular components,...",[ 38 ] \n
2,pathfinder\n,2017-03-01,Ability to share bases with other players New ...,[ 39 ] [ 40 ] [ 41 ] \n
3,atlas rises\n,2017-08-01,Addition of new narrative branch to main game'...,[ 43 ] [ 42 ] \n
4,next\n,2018-07-01,Added support for Xbox One and WeChat Full ...,[ 44 ] [ 45 ] [ 46 ] [ 47 ] [ 48 ] [ 49 ] [ 50...
5,the abyss\n,2018-10-01,Expanded aquatic biomes features include new c...,[ 52 ] \n
6,visions\n,2018-11-01,"Expansion of biome types, with introduction of...",[ 53 ] [ 54 ] \n
7,beyond\n,2019-08-01,Expansion of multiplayer experience with large...,[ 55 ] [ 56 ] [ 57 ] [ 58 ] [ 59 ] [ 60 ] [ 61...
8,synthesis\n,2019-11-01,Expanded inventory systems Starship upgrading \n,[ 63 ] [ 64 ] \n
9,living ship\n,2020-02-01,Addition of sentient living ships along with n...,[ 65 ] \n
10,exo mech\n,2020-04-01,Addition of mechanized suits that can be used ...,[ 66 ] \n


In [22]:
modWiki = df_series_strip(modWiki,target={"Title":r"\n"},compare=True)
modWiki = df_series_strip(modWiki,target={"Title":r"the "},compare=True)

Before vs after
1     foundation\n
2     pathfinder\n
3    atlas rises\n
4           next\n
5      the abyss\n
Name: Title, dtype: object
1     foundation
2     pathfinder
3    atlas rises
4           next
5      the abyss
Name: Title, dtype: object


Before vs after
5      the abyss
29    the cursed
Name: Title, dtype: object
5      abyss
29    cursed
Name: Title, dtype: object




In [23]:
modWiki = modWiki.drop(columns=['Notes', 'Ref'])
modWiki

,Title,Date
1,foundation,2016-11-01
2,pathfinder,2017-03-01
3,atlas rises,2017-08-01
4,next,2018-07-01
5,abyss,2018-10-01
6,visions,2018-11-01
7,beyond,2019-08-01
8,synthesis,2019-11-01
9,living ship,2020-02-01
10,exo mech,2020-04-01


# Attempt to combine data from scraped patch notes and wikipedia to fill in missing dates

In [24]:
(modDF[["Title"]].loc[modDF["Date"].isnull()==True].reset_index(drop=True)).eq((modWiki.reset_index(drop=True)))

,Date,Title
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
5,False,True
6,False,True
7,False,True
8,False,True
9,False,True


In [25]:
combiDF = modDF.merge(modWiki,how="outer",on="Title",suffixes=(None,"_wiki"))
combiDF

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Expedition,Date_wiki
0,abyss,NaT,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/,False,2018-10-01
1,adrift,2024-05-29,"'Expedition Thirteen: Adrift'\n'May 29, 2024'\...",True,False,False,https://www.nomanssky.com//2024/05/expedition-...,True,NaT
2,adrift patch 4.71,2024-05-31,"""Adrift Patch 4.71 - No Man's Sky""\n'Menu'\n'L...",False,True,False,https://www.nomanssky.com//2024/05/adrift-4-71/,False,NaT
3,adrift patch 4.71.1,2024-05-31,"""Adrift Patch 4.71.1 - No Man's Sky""\n'Menu'\n...",False,True,False,https://www.nomanssky.com//2024/05/adrift-4-71-1/,False,NaT
4,adrift patch 4.72,2024-06-04,"""Adrift Patch 4.72 - No Man's Sky""\n'Menu'\n'L...",False,True,False,https://www.nomanssky.com//2024/06/adrift-4-72/,False,NaT
...,...,...,...,...,...,...,...,...,...
236,worlds part ii 5.54,2025-02-06,"'Worlds Part II - 5.54'\n'February 06, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT
237,worlds part ii 5.55,2025-02-12,"'Worlds Part II - 5.55'\n'February 12, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT
238,worlds part ii 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT
239,worlds part ii 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT


In [26]:
combiDF.loc[combiDF["Date"].isnull()==True]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Expedition,Date_wiki
0,abyss,NaT,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/,False,2018-10-01
8,atlas rises,NaT,'Patch notes'\n'Generation'\n'Increased variet...,True,False,False,https://www.nomanssky.com//atlas-rises-update,False,2017-08-01
16,beacon,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-01
17,beyond,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//beyond-update/,False,2019-08-01
46,companions,NaT,'Buy now on'\n'More purchase options »'\n'3.2 ...,True,False,False,https://www.nomanssky.com//companions-update/,False,2021-02-01
61,desolation,NaT,'Buy now'\n'More purchase options »'\n'2.6 Pat...,True,False,False,https://www.nomanssky.com//desolation-update/,False,2020-07-01
75,echoes,NaT,'Buy now on'\n'More purchase options »'\n'4.4 ...,True,False,False,https://www.nomanssky.com//echoes-update/,False,2023-08-01
83,endurance,NaT,'Buy now on'\n'More purchase options »'\n'Out ...,True,False,False,https://www.nomanssky.com//endurance-update/,False,2022-07-01
89,exo mech,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//exo-mech-update/,False,2020-04-01
92,expeditions,NaT,'Buy now on'\n'More purchase options »'\n'3.3 ...,True,False,False,https://www.nomanssky.com//expeditions-update/,True,2021-03-01


In [27]:
combiDF.loc[(combiDF["Date"].isnull()==True),"Date"] = combiDF["Date_wiki"]
combiDF

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Expedition,Date_wiki
0,abyss,2018-10-01,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/,False,2018-10-01
1,adrift,2024-05-29,"'Expedition Thirteen: Adrift'\n'May 29, 2024'\...",True,False,False,https://www.nomanssky.com//2024/05/expedition-...,True,NaT
2,adrift patch 4.71,2024-05-31,"""Adrift Patch 4.71 - No Man's Sky""\n'Menu'\n'L...",False,True,False,https://www.nomanssky.com//2024/05/adrift-4-71/,False,NaT
3,adrift patch 4.71.1,2024-05-31,"""Adrift Patch 4.71.1 - No Man's Sky""\n'Menu'\n...",False,True,False,https://www.nomanssky.com//2024/05/adrift-4-71-1/,False,NaT
4,adrift patch 4.72,2024-06-04,"""Adrift Patch 4.72 - No Man's Sky""\n'Menu'\n'L...",False,True,False,https://www.nomanssky.com//2024/06/adrift-4-72/,False,NaT
...,...,...,...,...,...,...,...,...,...
236,worlds part ii 5.54,2025-02-06,"'Worlds Part II - 5.54'\n'February 06, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT
237,worlds part ii 5.55,2025-02-12,"'Worlds Part II - 5.55'\n'February 12, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT
238,worlds part ii 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT
239,worlds part ii 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,False,NaT


In [28]:
combiDF.loc[combiDF["Date"].isnull()==True]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Expedition,Date_wiki
212,switch,NaT,NaN,True,False,True,https://www.nomanssky.com//switch-update/,False,NaT


In [31]:
combiDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Title           241 non-null    object        
 1   Date            240 non-null    datetime64[ns]
 2   Notes           235 non-null    object        
 3   Patch update    239 non-null    object        
 4   Bug fix update  239 non-null    object        
 5   Scrape error    239 non-null    object        
 6   Patch page url  239 non-null    object        
 7   Expedition      239 non-null    object        
 8   Date_wiki       32 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(7)
memory usage: 17.1+ KB


In [33]:
combiDF = combiDF.drop_duplicates()
combiDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Title           241 non-null    object        
 1   Date            240 non-null    datetime64[ns]
 2   Notes           235 non-null    object        
 3   Patch update    239 non-null    object        
 4   Bug fix update  239 non-null    object        
 5   Scrape error    239 non-null    object        
 6   Patch page url  239 non-null    object        
 7   Expedition      239 non-null    object        
 8   Date_wiki       32 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(7)
memory usage: 17.1+ KB


In [36]:
combiDF = combiDF.drop(columns=["Date_wiki"])
combiDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Title           241 non-null    object        
 1   Date            240 non-null    datetime64[ns]
 2   Notes           235 non-null    object        
 3   Patch update    239 non-null    object        
 4   Bug fix update  239 non-null    object        
 5   Scrape error    239 non-null    object        
 6   Patch page url  239 non-null    object        
 7   Expedition      239 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 15.2+ KB


# Save the data

In [37]:
savePath = "./Data/"
saveFile = "NMS_patch_notes_full.xlsx"

tmp = ""
for savePathChunk in savePath.split("/")[:-1]:
    tmp += savePathChunk + "/"
    try:
        os.mkdir(tmp)
    except:
        pass

with pd.ExcelWriter(path=savePath+saveFile,mode="w") as writer:
    combiDF.to_excel(writer)